In [65]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

project ="myproject"
query = "select * from `firemandb.compiledfull` limit 10000"
df = pd.read_gbq(query,project_id=project,dialect='standard')

columns_to_parse = list(df.select_dtypes(include=['object']).columns)

for cparse in columns_to_parse:
  df[cparse] = pd.to_datetime(df[cparse])
  
for cparse in columns_to_parse:
  cday = 'day_{}'.format(cparse)
  cmonth = 'month_{}'.format(cparse)
  cyear = 'year_{}'.format(cparse)
  
  df[cday] = df[cparse].dt.day
  df[cmonth] = df[cparse].dt.month
  df[cyear] = df[cparse].dt.year
  
df = df.drop(['EventDate','DateRange'],axis=1)
  

y = df['IncendieBatiment_TrueFalse']
x = df.drop(['IncendieBatiment_TrueFalse'], axis=1)

X_train , X_test,y_train, y_test = train_test_split( 
            x, y , test_size=0.33, random_state= 42 )



Requesting query... ok.
Job ID: 21fb9b7e-4f39-4599-b2b2-8e9cda50309c
Query running...
Query done.
Cache hit.

Retrieving results...
Got 10000 rows.

Total time taken 203.55 s.
Finished at 2019-03-05 01:08:54.


In [69]:
from sklearn import tree
from sklearn.metrics import accuracy_score

clf = tree.DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=2, min_samples_leaf=5 )


clf = clf.fit(X_train,y_train)


y_pred = clf.predict(X_test)

In [71]:
print("current accuracy score:{}".format( accuracy_score(y_test,y_pred)*100))

current accuracy score:100.0


In [ ]:
#!gsutil cp compiledfullfixed.csv gs://projectcsv/compiledfullfixed.parquet.gzip
!gsutil cp ProjectML.ipynb gs://projectcsv/ProjectML.ipynb